In [1]:
## Basic imports
import sys
import os
import time

# WARNING: These global declarations cause the parallel implementation to 
# crash when executed on Windows
connections = None
networks = None
flowdepthvel = None

from sys import platform
if platform == "linux" or platform == "linux2":
    pass
elif platform == "darwin":
    pass
elif platform == "win32":
    print('The parallel version of compute_nhd_routing.py will not execute as currently')
    print('written due to the lack of a fork() capability in the windows OS.')
    print('For parallel execution, please us a *nix OS.')
    print('\nexiting...')
    sys.exit()
    # Some useful references:
    # https://stackoverflow.com/questions/985281/what-is-the-closest-thing-windows-has-to-fork/985525#985525
    # https://stackoverflow.com/questions/8220108/how-do-i-check-the-operating-system-in-python
    # https://stackoverflow.com/questions/6596617/python-multiprocess-diff-between-windows-and-linux

ENV_IS_CL = False
if ENV_IS_CL: root = '/content/wrf_hydro_nwm_public/trunk/NDHMS/dynamic_channel_routing/'
elif not ENV_IS_CL: 
    sys.setrecursionlimit(4000)
#     root = os.path.dirname(os.path.dirname(os.path.dirname(os.path.abspath(__file__))))
    root = os.path.dirname(os.path.dirname(os.path.abspath('')))
    sys.path.append(os.path.join(root, r'src', r'python_framework'))
    sys.path.append(os.path.join(root, r'src', r'python_routing')) #by Dong Ha
    fortran_source_dir = os.path.join(root, r'src', r'fortran_routing', r'mc_pylink_v00', r'MC_singleSeg_singleTS')
    sys.path.append(fortran_source_dir)

## F2PY Muskingum Cunge module
mc_f2py= True
if mc_f2py:
    try:
        import subprocess

        f2py_call = []
        f2py_call.append(r'f2py3')
        f2py_call.append(r'-c')
        f2py_call.append(r'varSingleSegStime_f2py_noIC_clean.f90')
        f2py_call.append(r'MCsingleSegStime_f2py_noIC_clean.f90')
        f2py_call.append(r'-m')
        f2py_call.append(r'mc_sseg_stime_noIC')
        subprocess.run(f2py_call, cwd=r'../fortran_routing/mc_pylink_v00/MC_singleSeg_singleTS',
                       stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)
    except Exception as e:
        print(e)

## network and reach utilities
import nhd_network_utilities as nnu
import nhd_reach_utilities as nru
import channel_routing_tools as crt

## Muskingum Cunge
import numpy as np

def mc_route_network(
          terminal_segment = None
        , network = None
        , supernetwork_data = None
        , nts = None
        , dt_mc= None
        , latflow= None
#         , connections = None
        , verbose = False
        , debuglevel = 0
        ):

    global connections
    global flowdepthvel 

    if verbose: print(f"Executing simulation on network {terminal_segment} beginning with streams of order {network['maximum_reach_seqorder']}")

    ##------------------------------------------------------------------------
    ## Compute ordered_reaches and last_segment_reach for all the segments of
    ## reach referenced by head_segment
    ##------------------------------------------------------------------------
    ordered_reaches = {}
    # Start: By Dong Ha
    last_segment_reach={}
    #End    
    for head_segment, reach in network['reaches'].items():
        if reach['seqorder'] not in ordered_reaches:
            ordered_reaches.update({reach['seqorder']:[]}) #TODO: Should this be a set/dictionary?
        ordered_reaches[reach['seqorder']].append([head_segment, reach])
        # Start: By Dong Ha
        seg_list=list(reach['segments_list'])
        lastseg=seg_list[0]
        last_segment_reach[head_segment]= lastseg
        # End 

    ##-------------------------------------------
    ## From jorder= maximum_reach_order to zero
    ##     From each reach that has jorder
    ##         run mc routing of a chosen scheme
    ##
    ## Output-> flowdepthvel
    ##--------------------------------------------
    #Note that not networks but network that takes networks values corresponding to 'terminal_segment' keys.
    #Also, network here takes 'terminal_segment' key only at the lower end of a network.      
    for x in range(network['maximum_reach_seqorder'],-1,-1):    
        
        for head_segment, reach in ordered_reaches[x]:       
#             print(latflow)
            ## START: run MC over each segment over entire timesteps
            crt.mc_tlp_over_seg(
                            connections= connections 
                            ,supernetwork_data= supernetwork_data
                            ,reach= reach
                            ,last_segment_reach= last_segment_reach
                            ,nts= nts
                            ,dt_mc= dt_mc
                            ,latflow= latflow
                            ,flowdepthvel= flowdepthvel
                            )            
            ## END: MC-computed results for flow, vel, and depth are stored in 'flowdepthvel'
           
            ## Start: Only for printing results
            seg_list=list(reach['segments_list'])
            seg_list=seg_list[::-1] #to reversed order
            ncomp= len(reach['segments_list'])
            for seg in range(0,ncomp):  
                segID= seg_list[seg]
                for ts in range (0,nts):
                    print(f"ts {ts} head_segment {head_segment} segINDEX {seg} segID {segID} \
                            FNL:ql qdc vel depth {flowdepthvel[segID]['qlat'][ts]} {flowdepthvel[segID]['flow'][ts]} \
                            {flowdepthvel[segID]['vel'][ts]} {flowdepthvel[segID]['depth'][ts]}")  
            with open(r"./output/flowdepthvel.txt","a") as mc_result:
                for seg in range(0,ncomp):  
                    segID= seg_list[seg]
                    for ts in range (0,nts):
                        mc_result.write("%s %s %s %s %s %s\n" % (ts, segID, flowdepthvel[segID]['qlat'][ts], 
                            flowdepthvel[segID]['flow'][ts], flowdepthvel[segID]['vel'][ts], flowdepthvel[segID]['depth'][ts]))        
              ## End  
                          
def main():

    global connections
    global networks
    global flowdepthvel

    verbose = True
    debuglevel = 0
    showtiming = True

    test_folder = os.path.join(root, r'test')
    geo_input_folder = os.path.join(test_folder, r'input', r'geo', r'Channels')

    #TODO: Make these commandline args
#     supernetwork= 'Pocono_TEST1'
    supernetwork= 'Pocono_TEST2'
    """##NHD Subset (Brazos/Lower Colorado)"""
#     supernetwork = 'Brazos_LowerColorado_ge5'
    """##NHD CONUS order 5 and greater"""
    # supernetwork = 'CONUS_ge5'
    """These are large -- be careful"""
#     supernetwork = 'Mainstems_CONUS'
    # supernetwork = 'CONUS_FULL_RES_v20'
    # supernetwork = 'CONUS_Named_Streams' #create a subset of the full resolution by reading the GNIS field
    # supernetwork = 'CONUS_Named_combined' #process the Named streams through the Full-Res paths to join the many hanging reaches

    if verbose: print('creating supernetwork connections set')
    if showtiming: start_time = time.time()
    #STEP 1
    supernetwork_data, supernetwork_values = nnu.set_networks(
        supernetwork = supernetwork
        , geo_input_folder = geo_input_folder
        , verbose = False
        # , verbose = verbose
        , debuglevel = debuglevel
        )
    if verbose: print('supernetwork connections set complete')
    if showtiming: print("... in %s seconds." % (time.time() - start_time))

    #STEP 2
    if showtiming: start_time = time.time()
    if verbose: print('organizing connections into networks and reaches ...')
    networks = nru.compose_reaches(
        supernetwork_values
        , verbose = False
        # , verbose = verbose
        , debuglevel = debuglevel
        , showtiming = showtiming
        )
    if verbose: print('reach organization complete')
    if showtiming: print("... in %s seconds." % (time.time() - start_time))
    

    #STEP 3: Route NHD streamflow
    if showtiming: start_time = time.time()
    executiontype = 'serial' # 'parallel'

    if verbose: print('executing serial computation on ordered reaches ...')
    connections = supernetwork_values[0]

    nts = 144  # number fof timestep = 1140 * 60(model timestep) = 86400 = day
    dt_mc= 300.0 # time interval for MC
    
    # Lateral flow
    ## test 1. Take lateral flow from wrf-hydro output from Pocono Basin
    qlcol=54
    qlrow=145
    ql=np.zeros((qlrow,qlcol))
    ql_input_folder= '../../test/input/geo/Channels/PoconoSampleData2/Pocono_ql_testsamp1_nwm_mc.txt' 
    print(ql_input_folder)
    for j in range(0, qlcol):
#         ql[0,j]=np.loadtxt("./input/Pocono_ql_testsamp1_nwm_mc.txt", max_rows=1,usecols=(j+2)) 
#         ql[1:,j]=np.loadtxt("./input/Pocono_ql_testsamp1_nwm_mc.txt", skiprows=1, usecols=(j+2))
        ql[0,j]=np.loadtxt(ql_input_folder, max_rows=1,usecols=(j+2)) 
        ql[1:,j]=np.loadtxt(ql_input_folder, skiprows=1, usecols=(j+2))
    latflow={}
    for j in range(0,qlcol):
        latflow.update({ql[0,j]:{'lateralflow':ql[1:,j]}})
       
    flowdepthvel = {connection:{'flow':np.zeros(nts)
                                , 'depth':np.zeros(nts)
                                , 'vel':np.zeros(nts)
                                , 'qlat':np.zeros(nts)}
                               for connection in connections}  

    if executiontype == 'serial':
        iter=0
        for terminal_segment, network in networks.items():
            if showtiming: network_start_time = time.time()
#             iter=iter+1
#             print(f"terminal_segment iter-> {iter}")

            ## run MC to route network 
            mc_route_network(
                  terminal_segment = terminal_segment
                , network = network
                , supernetwork_data = supernetwork_data
                , nts = nts
                , dt_mc= dt_mc
                , latflow= latflow                
#         , connections = None
                , verbose = False
                , debuglevel = 0
            )
        
#             if verbose: print(f'{terminal_segment} completed')
#             if showtiming: print("... in %s seconds." % (time.time() - network_start_time))
        
#     if verbose: print('ordered reach computation complete')
#     if showtiming: print("... in %s seconds." % (time.time() - start_time))

if __name__ == '__main__':
    main()


creating supernetwork connections set
supernetwork connections set complete
... in 0.031482696533203125 seconds.
organizing connections into networks and reaches ...
reach organization complete
... in 0.0008194446563720703 seconds.
executing serial computation on ordered reaches ...
../../test/input/geo/Channels/PoconoSampleData2/Pocono_ql_testsamp1_nwm_mc.txt
ts 0 head_segment 4185243 segINDEX 0 segID 4185243                             FNL:ql qdc vel depth 0.9937999844551086 0.20244789123535156                             0.1462530791759491 0.012524466961622238
ts 1 head_segment 4185243 segINDEX 0 segID 4185243                             FNL:ql qdc vel depth 0.9937999844551086 0.4985886514186859                             0.23942650854587555 0.02665754221379757
ts 2 head_segment 4185243 segINDEX 0 segID 4185243                             FNL:ql qdc vel depth 0.9937999844551086 0.7377896904945374                             0.33711010217666626 0.04545453190803528
ts 3 head_segment 

ts 89 head_segment 4186117 segINDEX 0 segID 4186117                             FNL:ql qdc vel depth 0.033399999141693115 0.4448390603065491                             0.39730000495910645 0.3595777153968811
ts 90 head_segment 4186117 segINDEX 0 segID 4186117                             FNL:ql qdc vel depth 0.033399999141693115 0.43466296792030334                             0.3943686783313751 0.35483500361442566
ts 91 head_segment 4186117 segINDEX 0 segID 4186117                             FNL:ql qdc vel depth 0.033399999141693115 0.42568275332450867                             0.3917377293109894 0.35060691833496094
ts 92 head_segment 4186117 segINDEX 0 segID 4186117                             FNL:ql qdc vel depth 0.033399999141693115 0.4178833067417145                             0.3894180655479431 0.34690138697624207
ts 93 head_segment 4186117 segINDEX 0 segID 4186117                             FNL:ql qdc vel depth 0.033399999141693115 0.4111896753311157                          

ts 143 head_segment 4185481 segINDEX 0 segID 4185481                             FNL:ql qdc vel depth 0.013000000268220901 0.013163018971681595                             0.20900528132915497 0.0438276045024395
ts 0 head_segment 4185481 segINDEX 1 segID 4186149                             FNL:ql qdc vel depth 2.138700008392334 0.06044074147939682                             0.04952102154493332 0.01028567273169756
ts 1 head_segment 4185481 segINDEX 1 segID 4186149                             FNL:ql qdc vel depth 2.138700008392334 0.21359489858150482                             0.08561212569475174 0.0236799456179142
ts 2 head_segment 4185481 segINDEX 1 segID 4186149                             FNL:ql qdc vel depth 2.138700008392334 0.43534213304519653                             0.12308688461780548 0.041494324803352356
ts 3 head_segment 4185481 segINDEX 1 segID 4186149                             FNL:ql qdc vel depth 2.138700008392334 0.7293009757995605                             0.1640

ts 105 head_segment 4186147 segINDEX 0 segID 4186147                             FNL:ql qdc vel depth 0.05009999871253967 3.4027037620544434                             0.8598594069480896 1.0226736068725586
ts 106 head_segment 4186147 segINDEX 0 segID 4186147                             FNL:ql qdc vel depth 0.05009999871253967 3.386415481567383                             0.8592100739479065 1.0212432146072388
ts 107 head_segment 4186147 segINDEX 0 segID 4186147                             FNL:ql qdc vel depth 0.05009999871253967 3.3711910247802734                             0.8586016297340393 1.0199041366577148
ts 108 head_segment 4186147 segINDEX 0 segID 4186147                             FNL:ql qdc vel depth 0.03840000182390213 3.3528411388397217                             0.8578661680221558 1.0182865858078003
ts 109 head_segment 4186147 segINDEX 0 segID 4186147                             FNL:ql qdc vel depth 0.03840000182390213 3.3350980281829834                             0.85

ts 112 head_segment 4186133 segINDEX 0 segID 4186133                             FNL:ql qdc vel depth 0.5401999950408936 0.5626459717750549                             0.6155838966369629 0.26323792338371277
ts 113 head_segment 4186133 segINDEX 0 segID 4186133                             FNL:ql qdc vel depth 0.5401999950408936 0.5606182217597961                             0.6148303747177124 0.26268407702445984
ts 114 head_segment 4186133 segINDEX 0 segID 4186133                             FNL:ql qdc vel depth 0.5401999950408936 0.5587753653526306                             0.6141437888145447 0.2621799409389496
ts 115 head_segment 4186133 segINDEX 0 segID 4186133                             FNL:ql qdc vel depth 0.5401999950408936 0.5571002960205078                             0.6135182976722717 0.26172110438346863
ts 116 head_segment 4186133 segINDEX 0 segID 4186133                             FNL:ql qdc vel depth 0.5401999950408936 0.5555774569511414                             0.612

ts 64 head_segment 4185571 segINDEX 0 segID 4185571                             FNL:ql qdc vel depth 9.999999747378752e-05 0.6793961524963379                             0.82440185546875 0.28296035528182983
ts 65 head_segment 4185571 segINDEX 0 segID 4185571                             FNL:ql qdc vel depth 9.999999747378752e-05 0.6656302809715271                             0.8188503980636597 0.279619425535202
ts 66 head_segment 4185571 segINDEX 0 segID 4185571                             FNL:ql qdc vel depth 9.999999747378752e-05 0.654596209526062                             0.8143383264541626 0.27691924571990967
ts 67 head_segment 4185571 segINDEX 0 segID 4185571                             FNL:ql qdc vel depth 9.999999747378752e-05 0.6458290219306946                             0.8107123374938965 0.27475935220718384
ts 68 head_segment 4185571 segINDEX 0 segID 4185571                             FNL:ql qdc vel depth 9.999999747378752e-05 0.6389053463935852                            

ts 142 head_segment 2743396 segINDEX 0 segID 2743396                             FNL:ql qdc vel depth 0.0 0.2498837113380432                             0.04539019986987114 1.19648015499115
ts 143 head_segment 2743396 segINDEX 0 segID 2743396                             FNL:ql qdc vel depth 0.0 0.24757157266139984                             0.04530433565378189 1.1922279596328735
